# Домашнее задание 4
## Text Normalization 

deadline: 15 декабря 2019, 23:59

В этом домашнем задании вы будете работать с корпусом соревнования по нормализации текстов на русском языке. 

Ссылка на соревнование:
https://www.kaggle.com/c/text-normalization-challenge-russian-language

Корпус (train-test split) доступен там же, на kaggle. Кроме того, kaggle проверяет результаты на тестовом множестве. Пример сабмита в файле: ru_sample_submission_2. 

Задача заключается в том, привести исходный текст (колонку before) в нормализованную форму (колонка after). Дополнительно известны классы токенов (колонка class), общее число классов – 15. В тестовом множестве классы токенов отсутствуют. 

Корпус состоит из предложений на русском языке и их нормализованных аналогов. Примеры продемонстрированы на kaggle.

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 3-х человек.
2. Домашнее задание сдается через anytask, инвайты будут дополнительно высланы.
3. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо ipython-тетрадке. 
4. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
5. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
6. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
7. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 

In [1]:
import os

In [2]:
DATA_PREFFIX = 'data/'
TEST_1_DATA_PATH = DATA_PREFFIX + 'ru_test.csv'
TEST_2_DATA_PATH = DATA_PREFFIX + 'ru_test_2.csv'
TRAIN_DATA_PATH = DATA_PREFFIX + 'ru_train.csv'
SIMPLE_DATA_TRAIN_PATH = DATA_PREFFIX + 'simple_data_train.tsv'
SIMPLE_DATA_TEST_PATH = DATA_PREFFIX + 'simple_data_test.tsv'
BACKUP_PREFFIX = 'backup/'
os.makedirs(BACKUP_PREFFIX, exist_ok=True)

In [3]:
import pandas as pd
import numpy as np
import torch
import torchtext
from tqdm import tqdm_notebook as tqdm
import pickle
import gc

In [4]:
df = pd.read_csv(TRAIN_DATA_PATH)

In [5]:
df[:10]

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,По,По
1,0,1,PLAIN,состоянию,состоянию
2,0,2,PLAIN,на,на
3,0,3,DATE,1862 год,тысяча восемьсот шестьдесят второй год
4,0,4,PUNCT,.,.
5,1,0,PLAIN,Оснащались,Оснащались
6,1,1,PLAIN,латными,латными
7,1,2,PLAIN,рукавицами,рукавицами
8,1,3,PLAIN,и,и
9,1,4,PLAIN,сабатонами,сабатонами


## Часть 1. [1 балл] Эксплоративный анализ

1. Найдите примеры каждого класса и опишите, по какой логике проведена нормализация токенов разных классов. 
2. В каких случаях токены класса PLAIN подвергаются нормализации? 
3. Напишите правила для нормализации токенов класса ORDINAL. 

#### Пункт 1.

In [6]:
classes = set(df['class'].values)
print(*classes)

CARDINAL MONEY MEASURE TELEPHONE PUNCT VERBATIM FRACTION DECIMAL ELECTRONIC DATE TIME LETTERS DIGIT PLAIN ORDINAL


In [7]:
df[df['class'] == 'DIGIT'].head()

,sentence_id,token_id,class,before,after
2524,178,4,DIGIT,06,ноль шесть
9254,683,7,DIGIT,007,ноль ноль семь
22460,1667,4,DIGIT,00,ноль ноль
23209,1722,8,DIGIT,0171,ноль один семь один
36632,2680,22,DIGIT,09,ноль девять


* DIGIT состоит из цифр, переходит в последовательность слов 

In [8]:
df[df['class'] == 'MONEY'].head()

,sentence_id,token_id,class,before,after
12028,883,5,MONEY,194 млн долл.,ста девяноста четырех миллионов долларов
18226,1345,10,MONEY,2 миллионов долларов,двух миллионов долларов
20298,1504,9,MONEY,10 000 рублей,десять тысяч рублей
22832,1696,9,MONEY,200 миллионов рублей,двести миллионов рублей
24820,1835,4,MONEY,250 рублей,двести пятьдесят рублей


* MONEY состоит из числа суммы денег и единицы измерения, переходит в соотв последовательность слов

In [9]:
df[df['class'] == 'LETTERS'].head()

,sentence_id,token_id,class,before,after
92,6,8,LETTERS,TV,t v
186,12,9,LETTERS,РСФСР,р с ф с р
301,24,1,LETTERS,СПб,с п б
340,26,1,LETTERS,СПб,с п б
346,26,7,LETTERS,В. А.,в а


* LETTERS состоит из аббревиатур, переходит в маленькие буквы, разделенные пробелами

In [10]:
df[df['class'] == 'CARDINAL'].head()

,sentence_id,token_id,class,before,after
137,9,9,CARDINAL,254,двести пятьдесят четыре
272,21,7,CARDINAL,2014,две тысячи четырнадцать
275,21,10,CARDINAL,12,двенадцать
304,24,4,CARDINAL,2014,две тысячи четырнадцать
343,26,4,CARDINAL,2011,две тысячи одиннадцать


* CARDINAL состоит из чисел, переходит в слова, обозначающие эти числа

In [11]:
df[df['class'] == 'FRACTION'].head()

,sentence_id,token_id,class,before,after
3578,259,17,FRACTION,1883/1884,тысяча восемьсот восемьдесят три тысяча восемь...
15218,1121,7,FRACTION,653/26,шестьсот пятьдесят три двадцать шестых
23441,1735,11,FRACTION,3/16,трех шестнадцатых
24061,1781,5,FRACTION,2/3,две третьих
28159,2075,17,FRACTION,2014/15,две тысячи четырнадцать пятнадцатых


* FRACTION состоит из дробного числа, переходит в словарную запись дроби

In [12]:
df[df['class'] == 'TELEPHONE'].head()

,sentence_id,token_id,class,before,after
576,41,2,TELEPHONE,978-5-86007-700-3,девятьсот семьдесят восемь sil пять sil восемь...
4100,302,15,TELEPHONE,973-96857-7-3,девятьсот семьдесят три sil девятьсот шестьдес...
5530,418,1,TELEPHONE,0-87778-245-8,ноль sil восемьсот семьдесят семь семьдесят во...
5663,426,9,TELEPHONE,0-446-69334-0,ноль sil четыреста сорок шесть sil шестьсот де...
5887,443,10,TELEPHONE,5-9 221-0 440-3,пять sil девять sil двести двадцать один sil н...


* TELEPHONE состоит из номера телефона с дефисами, переходит в запись на русском языке, вместо дефисов в записи появляются слова 'sil'

In [13]:
df[df['class'] == 'DIGIT'].head()

,sentence_id,token_id,class,before,after
2524,178,4,DIGIT,06,ноль шесть
9254,683,7,DIGIT,007,ноль ноль семь
22460,1667,4,DIGIT,00,ноль ноль
23209,1722,8,DIGIT,0171,ноль один семь один
36632,2680,22,DIGIT,09,ноль девять


* DIGIT состоит из цифр, несущих в себе лишь смысл "последовательность цифр"

In [14]:
df[df['class'] == 'PUNCT'].head()

,sentence_id,token_id,class,before,after
4,0,4,PUNCT,.,.
14,1,9,PUNCT,.,.
18,2,3,PUNCT,",",","
24,2,9,PUNCT,(,(
27,2,12,PUNCT,),)


* PUNCT состоит из знаков пунктуации, переходит в себя

In [15]:
df[df['class'] == 'TIME'].head()

,sentence_id,token_id,class,before,after
14366,1059,1,TIME,11:19:52,11:19:52
14615,1076,1,TIME,11:20:41,11:20:41
36235,2652,2,TIME,06:06,шесть часов шесть минут
45775,3347,2,TIME,15:03:28,15:03:28
57147,4195,10,TIME,02:33,два часа тридцать три минуты


* TIME состоит из времени, переходит в себя, когда точность до секунд, в последовательность слов, когда до минут

In [16]:
df[df['class'] == 'MEASURE'].head()

,sentence_id,token_id,class,before,after
365,27,12,MEASURE,61st,шестьдесят один стоун
412,30,13,MEASURE,31 минуту,тридцати одной минуту
474,34,12,MEASURE,500 км,пятисот километров
605,44,6,MEASURE,80 см.,восемьдесят сантиметров
793,55,18,MEASURE,480 с.,четыреста восемьдесят секунд


* MEASURE состоит из числа и единицы измерения, переходит в число словами и расшифровку этой единицы

In [17]:
df[df['class'] == 'VERBATIM'].head()

,sentence_id,token_id,class,before,after
60,4,4,VERBATIM,-,-
109,7,10,VERBATIM,兄,兄
110,7,11,VERBATIM,貴,貴
125,8,9,VERBATIM,-,-
212,16,7,VERBATIM,-,-


* VERBATIM состоит из специальных символов, переходит в себя

In [18]:
df[df['class'] == 'DECIMAL'].head()

,sentence_id,token_id,class,before,after
1292,91,10,DECIMAL,60 тысяч,шестьдесят тысяч
7237,542,7,DECIMAL,1 млн,одного миллиона
7685,575,16,DECIMAL,"1,2",одна целая и две десятых
7687,575,18,DECIMAL,"3,5",три целых и пять десятых
8712,640,16,DECIMAL,"7203,54",семь тысяч двести три целых и пятьдесят четыре...


* DECIMAL состоит из десятичных дробей, переходит в слова

In [19]:
df[df['class'] == 'DATE'].head()

,sentence_id,token_id,class,before,after
3,0,3,DATE,1862 год,тысяча восемьсот шестьдесят второй год
17,2,2,DATE,1811 года,тысяча восемьсот одиннадцатого года
85,6,1,DATE,12 февраля 2013,двенадцатого февраля две тысячи тринадцатого года
90,6,6,DATE,15 февраля 2013,пятнадцатого февраля две тысячи тринадцатого года
189,13,1,DATE,1905 года,тысяча девятьсот пятого года


* DATE состоит из дат, переходит в слова

In [20]:
df[df['class'] == 'ELECTRONIC'].head()

,sentence_id,token_id,class,before,after
1257,89,7,ELECTRONIC,ElvisPresleyBiography.com,э_trans л_trans в_trans и_trans с_trans п_tran...
3157,224,23,ELECTRONIC,About.com,э_trans б_trans а_trans у_trans т_trans точка ...
3883,284,0,ELECTRONIC,Billboard.com,б_trans и_trans л_trans б_trans о_trans р_tran...
5808,437,0,ELECTRONIC,THG.ru,t h g точка р_trans у_trans
9400,695,1,ELECTRONIC,2014-09-01.Charts.org.nz,две тысячи четырнадцать дефис ноль девять дефи...


* ELECTRONIC состоит из аглийских слов и чисел. Англ слова переходят в их транскрипцию, числа в их запись словами

In [21]:
df[df['class'] == 'PLAIN'].head()

,sentence_id,token_id,class,before,after
0,0,0,PLAIN,По,По
1,0,1,PLAIN,состоянию,состоянию
2,0,2,PLAIN,на,на
5,1,0,PLAIN,Оснащались,Оснащались
6,1,1,PLAIN,латными,латными


* PLAIN состоит из слов. Русские слова переходят в себя, англ слова в транскрипцию

In [22]:
df[df['class'] == 'ORDINAL'].head()

,sentence_id,token_id,class,before,after
53,3,17,ORDINAL,III,третьего
163,11,1,ORDINAL,1895,тысяча восемьсот девяносто пятом
164,11,2,ORDINAL,—1896,тысяча восемьсот девяносто шестом
485,35,5,ORDINAL,1796,тысяча семьсот девяносто шестого
487,35,7,ORDINAL,1802,тысяча восемьсот второй


* ORDINAL состоит из арабских или римских чисел, переходят в слова

Нормализация происходит так, чтобы Text-To-Speach системы могли произнести текст

#### Пункт 2.

In [23]:
MAX_SAMPLES = 10
samples_counter = 0
for i in range(df.shape[0]):
    cls = df['class'][i]
    before = df['before'][i]
    after = df['after'][i]
    if cls == 'PLAIN' and before != after:
        print('%-15s\t%-30s\t%-100s' % (cls, 'Before', 'After'))
        print('%-15s\t%-30s\t%-100s' % ('', df['before'][i], df['after'][i] ))
        print()
        samples_counter += 1
    if samples_counter == MAX_SAMPLES:
        break

PLAIN          	Before                        	After                                                                                               
               	Tiberius                      	т_trans и_trans б_trans е_trans р_trans и_trans у_trans с_trans                                     

PLAIN          	Before                        	After                                                                                               
               	Julius                        	д_trans ж_trans у_trans л_trans и_trans у_trans с_trans                                             

PLAIN          	Before                        	After                                                                                               
               	Pollienus                     	п_trans о_trans л_trans л_trans и_trans е_trans н_trans у_trans с_trans                             

PLAIN          	Before                        	After                                                         

Класс PLAIN преобразуется, если слово написано не на русском языке. Тогда оно преобразуется посредством транслитерации

#### Пункт 3.

In [24]:
MAX_SAMPLES = 50
samples_counter = 0
for i in range(df.shape[0]):
    cls = df['class'][i]
    before = df['before'][i]
    after = df['after'][i]
    if cls == 'ORDINAL' and before != after:
        print('%-15s\t%-30s\t%-100s' % (cls, 'Before', 'After'))
        print('%-15s\t%-30s\t%-100s' % ('', df['before'][i], df['after'][i] ))
        print()
        samples_counter += 1
    if samples_counter == MAX_SAMPLES:
        break

ORDINAL        	Before                        	After                                                                                               
               	III                           	третьего                                                                                            

ORDINAL        	Before                        	After                                                                                               
               	1895                          	тысяча восемьсот девяносто пятом                                                                    

ORDINAL        	Before                        	After                                                                                               
               	—1896                         	тысяча восемьсот девяносто шестом                                                                   

ORDINAL        	Before                        	After                                                         

ORDINAL        	Before                        	After                                                                                               
               	85-й                          	восемьдесят пятый                                                                                   

ORDINAL        	Before                        	After                                                                                               
               	XX                            	двадцатого                                                                                          

ORDINAL        	Before                        	After                                                                                               
               	II                            	второй                                                                                              

ORDINAL        	Before                        	After                                                         

*  Римские цифры переводятся в прилагательные (склонения мб зависят от контекста)

*  Когда после числа стоит -й/-я/-е/-х/-му, число переходит в прилагательное отвечающее на вопрос "который"/"которая"/"которые"/"в которых"/"которому"

*  Когда перед числом тире прилагательное отвечает на вопрос "в каком"

*  Когда число не окружено ничем оно переходит либо в прилагательное в зависящем от контекста склонении, либо остается числом. Скорее всего тоже зависит от контекста.

## Часть 2. [6 баллов]  seq2seq архитектуры
Имплементируйте несколько seq2seq архитектур. Энкодер получает на вход последовательность токенов before, декодер учится превращать их в токены after.
Энкодер и декодер работают на уровне символов, эмбеддинги символов инициализируются случайно (по аналогии с работами, в которых предложены нейросетевые модели исправления опечаток).

Эту часть задания рекомендуется выполнять с использованием allennlp (должно быть проще и удобнее).

1. [3 балла] LSTM encoder + LSTM decoder + три механизма внимания: скалярное произведение, аддитивное внимание и мультипликативное внимание (см. лекцию 6, слайд "подсчет весов attention")
2. [3 балла] Transformer

Используя автопровереку kaggle, оцените, как влияют параметры архитектуры на качество задачи.

[бонус] convolutional encoder + convolutional decoder

[бонус] pyramid LSTM (размер l+1 слоя в два раз меньше размера l, i-тый вход l+1 слоя – конкатенация выходов 2i и 2i+1)

In [25]:
from torch import nn
import itertools

import allennlp
from allennlp.data.dataset_readers.seq2seq import Seq2SeqDatasetReader
from allennlp.data.tokenizers.word_tokenizer import WordTokenizer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.instance import Instance

from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.iterators import BucketIterator

from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders.pytorch_seq2seq_wrapper import PytorchSeq2SeqWrapper
from allennlp.models.encoder_decoders import SimpleSeq2Seq
from allennlp.modules.seq2seq_encoders.stacked_self_attention import StackedSelfAttentionEncoder

from allennlp.modules.attention.additive_attention import AdditiveAttention
from allennlp.modules.attention.bilinear_attention import BilinearAttention
from allennlp.modules.attention.dot_product_attention import DotProductAttention


from allennlp.training.trainer import Trainer

from allennlp.predictors import SimpleSeq2SeqPredictor

from sklearn.model_selection import train_test_split

In [26]:
HIDDEN_DIM = 64
EMBEDDING_DIM = 256
MAX_DECODING_STEPS = 75
BATCH_SIZE=64
N_EPOCHS = 1
NUM_LAYERS = 2

MAX_DF = 10**6

In [27]:
def prepare_simple_dataset(df, target_file_path):
    data = pd.DataFrame(df[['before', 'after']].values)
    data.to_csv(target_file_path, sep='\t', index=False)

df_train, df_test = train_test_split(df[:min(MAX_DF, df.shape[0])], test_size=0.3, random_state=42)
prepare_simple_dataset(df_train, SIMPLE_DATA_TRAIN_PATH)
prepare_simple_dataset(df_test, SIMPLE_DATA_TEST_PATH)

In [28]:
import builtins
import functools

old_open = open
uopen = functools.partial(open, encoding='utf8')
builtins.open = uopen

In [29]:
reader = Seq2SeqDatasetReader(
    source_tokenizer=CharacterTokenizer(),
    target_tokenizer=CharacterTokenizer(),
    source_token_indexers={'tokens': SingleIdTokenIndexer()},
    target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})

dataset_train = reader.read(SIMPLE_DATA_TRAIN_PATH)
dataset_test = reader.read(SIMPLE_DATA_TEST_PATH)

700001it [00:26, 26633.05it/s]
300001it [00:09, 31496.27it/s]


In [30]:
builtins.open = old_open

In [31]:
vocab = Vocabulary.from_instances(dataset_train,
                                      min_count={'tokens': 3, 'target_tokens': 3})

embeddings = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                         embedding_dim=EMBEDDING_DIM)

source_embedder = BasicTextFieldEmbedder({"tokens": embeddings})

100%|████████████████████████████████████████████████████████████████| 700001/700001 [00:07<00:00, 95746.03it/s]


#### Пункт 1.

In [32]:
attentions = [None,
              DotProductAttention(),
              BilinearAttention(HIDDEN_DIM, HIDDEN_DIM),
              AdditiveAttention(HIDDEN_DIM, HIDDEN_DIM)
             ]

In [33]:
def make_model(attention_layer=None, embed_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM):    
    encoder = PytorchSeq2SeqWrapper(nn.LSTM(embed_dim, hidden_dim, batch_first=True, dropout=0.3, num_layers=NUM_LAYERS))

    model = SimpleSeq2Seq(vocab, source_embedder, encoder, MAX_DECODING_STEPS,
                              target_embedding_dim=embed_dim,
                              target_namespace='target_tokens',
                              attention=attention_layer)
    return model

models = [make_model(att) for att in attentions]
optimizers = [torch.optim.Adam(model.parameters()) for model in models]

In [34]:
iterator = BucketIterator(batch_size=BATCH_SIZE, sorting_keys=[("source_tokens", "num_tokens")])
iterator.index_with(vocab)

In [35]:
trainers = [Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=dataset_train,
                  validation_dataset=dataset_test,
                  num_epochs=1,
                  cuda_device=0) for (model, optimizer) in zip(models, optimizers)]

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


In [36]:
from IPython.display import clear_output


def train(model, trainer, epochs=N_EPOCHS, reader=reader, clear_out=True, only_examples=False):
    model.cuda()
    def tokens2text(tokens, start=0, end=None):
        tokens = list(tokens)
        end = len(tokens) if end is None else end
        tokens = tokens[start:end]
        text = map(str, tokens)
        text = ''.join(text)
        return text
    
    for i in range(epochs):
        predictor = SimpleSeq2SeqPredictor(model, reader)
        if clear_out and i == epochs - 1:
            clear_output()
        if not only_examples:
            trainer.train()
        if clear_out and i != epochs - 1:
            clear_output()
            
        predictor = SimpleSeq2SeqPredictor(model, reader)
        print('Epoch %d' % i)
        format_str = 'Before:\t\t%s\nTrueAfter\t%s\nPredAfter\t%s\n\n'
        max_samples = 50
        max_plain = 5
        samples_count = 0
        plain_count = 0
        for instance in dataset_test:
            x_data = list(instance.fields['source_tokens'].tokens)
            y_true = instance.fields['target_tokens'].tokens
            y_pred = predictor.predict_instance(instance)['predicted_tokens']
            if x_data == y_true:
                if plain_count < max_plain:
                    plain_count += 1
                else:
                    continue
            print(format_str % (tokens2text(x_data, 1, -1),
                                           tokens2text(y_true, 1, -1),
                                           tokens2text(y_pred)))
            samples_count += 1
            if samples_count == max_samples:
                break
        if only_examples:
            break
    model.cpu()

In [39]:
model_ind = 0
def train_model_or_load(model_ind):
    loaded = False
    path = BACKUP_PREFFIX + 'seq2seq_%d.mdl' % model_ind
    model = models[model_ind]
    if os.path.exists(path):
        model.load_state_dict(torch.load(path))
        #loaded = True
        
    train(model, trainers[model_ind], only_examples=loaded)    

In [38]:
print('Training model with attention:', attentions[model_ind].__class__)
train_model_or_load(model_ind)
gc.collect()
torch.cuda.empty_cache()

model_ind += 1

Epoch 0
Before:		0
TrueAfter	1
PredAfter	с


Before:		фантазию
TrueAfter	фантазию
PredAfter	фартррнл


Before:		на
TrueAfter	на
PredAfter	на


Before:		сан
TrueAfter	сан
PredAfter	сон


Before:		один
TrueAfter	один
PredAfter	однна


Before:		1 января 2014 года
TrueAfter	первого января две тысячи четырнадцатого года
PredAfter	дядоого гвоодо


Before:		коленчато
TrueAfter	коленчато
PredAfter	коменоать


Before:		2009 году
TrueAfter	две тысячи девятом году
PredAfter	две тысячи девятьс года


Before:		20
TrueAfter	двадцать
PredAfter	дведцать


Before:		cam
TrueAfter	к_trans а_trans м_trans
PredAfter	с_trans и_trans н_trans 


Before:		iPad
TrueAfter	а_trans й_trans п_trans а_trans д_trans
PredAfter	с_trans н_trans н_trans и_trans н_trans 


Before:		Maxim
TrueAfter	м_trans а_trans к_trans с_trans и_trans м_trans
PredAfter	м_trans о_trans н_trans и_trans и_trans н_trans 


Before:		York
TrueAfter	й_trans о_trans р_trans к_trans
PredAfter	с_trans р_trans н_trans н_trans 


Before:		Ф. И.
Tru

In [40]:
print('Training model with attention:', attentions[model_ind].__class__)
train_model_or_load(model_ind)
gc.collect()
torch.cuda.empty_cache()

model_ind += 1

loss: 0.4181 ||: 100%|████████████████████████████████████████████████████| 10938/10938 [16:28<00:00, 11.06it/s]
BLEU: 0.6064, loss: 0.2564 ||: 100%|████████████████████████████████████████| 4688/4688 [26:31<00:00,  2.95it/s]


Epoch 0
Before:		0
TrueAfter	1
PredAfter	ноль


Before:		фантазию
TrueAfter	фантазию
PredAfter	фантазае


Before:		на
TrueAfter	на
PredAfter	на


Before:		сан
TrueAfter	сан
PredAfter	сан


Before:		один
TrueAfter	один
PredAfter	один


Before:		1 января 2014 года
TrueAfter	первого января две тысячи четырнадцатого года
PredAfter	пятьдесят пять


Before:		коленчато
TrueAfter	коленчато
PredAfter	колтечани


Before:		2009 году
TrueAfter	две тысячи девятом году
PredAfter	две тысячи девять


Before:		20
TrueAfter	двадцать
PredAfter	двадцать девять


Before:		cam
TrueAfter	к_trans а_trans м_trans
PredAfter	к_trans а_trans н_trans с_trans


Before:		iPad
TrueAfter	а_trans й_trans п_trans а_trans д_trans
PredAfter	р_trans е_trans н_trans


Before:		Maxim
TrueAfter	м_trans а_trans к_trans с_trans и_trans м_trans
PredAfter	м_trans а_trans н_trans с_trans е_trans н_trans с_trans


Before:		York
TrueAfter	й_trans о_trans р_trans к_trans
PredAfter	т_trans о_trans л_trans а_trans н_trans с_trans


Bef

In [41]:
print('Training model with attention:', attentions[model_ind].__class__)
train_model_or_load(model_ind)
gc.collect()
torch.cuda.empty_cache()

model_ind += 1

loss: 0.0477 ||: 100%|████████████████████████████████████████████████████| 10938/10938 [29:16<00:00,  6.23it/s]
BLEU: 0.8894, loss: 0.0296 ||: 100%|████████████████████████████████████████| 4688/4688 [30:13<00:00,  2.58it/s]


Epoch 0
Before:		0
TrueAfter	1
PredAfter	ноль


Before:		фантазию
TrueAfter	фантазию
PredAfter	фантазию


Before:		на
TrueAfter	на
PredAfter	на


Before:		сан
TrueAfter	сан
PredAfter	сан


Before:		один
TrueAfter	один
PredAfter	один


Before:		1 января 2014 года
TrueAfter	первого января две тысячи четырнадцатого года
PredAfter	пятнадцатого августа две тысячи четырнадцатого года


Before:		коленчато
TrueAfter	коленчато
PredAfter	коленчато


Before:		2009 году
TrueAfter	две тысячи девятом году
PredAfter	две тысячи девятого году


Before:		20
TrueAfter	двадцать
PredAfter	двадцати


Before:		cam
TrueAfter	к_trans а_trans м_trans
PredAfter	к_trans а_trans м_trans


Before:		iPad
TrueAfter	а_trans й_trans п_trans а_trans д_trans
PredAfter	и_trans т_trans а_trans


Before:		Maxim
TrueAfter	м_trans а_trans к_trans с_trans и_trans м_trans
PredAfter	м_trans а_trans к_trans с_trans и_trans м_trans


Before:		York
TrueAfter	й_trans о_trans р_trans к_trans
PredAfter	я_trans р_trans к_trans


Before

In [42]:
print('Training model with attention:', attentions[model_ind].__class__)
train_model_or_load(model_ind)
gc.collect()
torch.cuda.empty_cache()

model_ind += 1

loss: 0.4350 ||: 100%|████████████████████████████████████████████████████| 10938/10938 [32:43<00:00,  5.57it/s]
BLEU: 0.8246, loss: 0.0858 ||: 100%|████████████████████████████████████████| 4688/4688 [32:45<00:00,  2.39it/s]


Epoch 0
Before:		0
TrueAfter	1
PredAfter	ноль


Before:		фантазию
TrueAfter	фантазию
PredAfter	фантазию


Before:		на
TrueAfter	на
PredAfter	на


Before:		сан
TrueAfter	сан
PredAfter	сан


Before:		один
TrueAfter	один
PredAfter	один


Before:		1 января 2014 года
TrueAfter	первого января две тысячи четырнадцатого года
PredAfter	двадцать первого года


Before:		коленчато
TrueAfter	коленчато
PredAfter	коленчато


Before:		2009 году
TrueAfter	две тысячи девятом году
PredAfter	две тысячи девятом году


Before:		20
TrueAfter	двадцать
PredAfter	двадцать


Before:		cam
TrueAfter	к_trans а_trans м_trans
PredAfter	к_trans а_trans н_trans


Before:		iPad
TrueAfter	а_trans й_trans п_trans а_trans д_trans
PredAfter	и_trans р_trans д_trans


Before:		Maxim
TrueAfter	м_trans а_trans к_trans с_trans и_trans м_trans
PredAfter	м_trans а_trans н_trans и_trans н_trans


Before:		York
TrueAfter	й_trans о_trans р_trans к_trans
PredAfter	я_trans р_trans о_trans с_trans


Before:		Ф. И.
TrueAfter	ф и
PredAfte

In [43]:
for i, model in enumerate(models):
    torch.save(model.state_dict(), BACKUP_PREFFIX + 'seq2seq_%d.mdl' % i)

#### Пункт 2.

In [44]:
transformer_encoder = StackedSelfAttentionEncoder(
    input_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    projection_dim=128,
    feedforward_hidden_dim=128,
    num_layers=1,
    num_attention_heads=8)
transformer_model = SimpleSeq2Seq(vocab, source_embedder, transformer_encoder, MAX_DECODING_STEPS,
                              target_embedding_dim=EMBEDDING_DIM,
                              target_namespace='target_tokens')
transformer_optimizer = torch.optim.Adam(transformer_model.parameters())
transformer_trainer = Trainer(model=transformer_model,
                  optimizer=transformer_optimizer,
                  iterator=iterator,
                  train_dataset=dataset_train,
                  validation_dataset=dataset_test,
                  num_epochs=1,
                  cuda_device=0)

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


In [45]:
def train_transformer_or_load():
    loaded = False
    path = BACKUP_PREFFIX + 'transformer.mdl'
    if os.path.exists(path):
        transformer_model.load_state_dict(torch.load(path))
        #loaded = True
        
    train(transformer_model, transformer_trainer, only_examples=loaded)    

In [46]:
train_transformer_or_load()
gc.collect()
torch.cuda.empty_cache()

loss: 0.8467 ||: 100%|████████████████████████████████████████████████████| 10938/10938 [16:52<00:00, 10.81it/s]
BLEU: 0.2946, loss: 0.7025 ||: 100%|████████████████████████████████████████| 4688/4688 [32:13<00:00,  2.43it/s]


Epoch 0
Before:		0
TrueAfter	1
PredAfter	пять


Before:		фантазию
TrueAfter	фантазию
PredAfter	фантрана


Before:		на
TrueAfter	на
PredAfter	на


Before:		сан
TrueAfter	сан
PredAfter	сам


Before:		один
TrueAfter	один
PredAfter	одни


Before:		1 января 2014 года
TrueAfter	первого января две тысячи четырнадцатого года
PredAfter	тысяча девятьсот девятьсот девятьсот девятьсот девятьсот девятьсот девятьсо


Before:		коленчато
TrueAfter	коленчато
PredAfter	которона


Before:		2009 году
TrueAfter	две тысячи девятом году
PredAfter	две тысячи девятьсот девятьсот девятьсот девятьсот девятьсот девятьсот девя


Before:		20
TrueAfter	двадцать
PredAfter	двадцать тридцать тридцать тридцать тридцать тридцать тридцать тридцать три


Before:		cam
TrueAfter	к_trans а_trans м_trans
PredAfter	к_trans а_trans р_trans т_trans е_trans р_trans т_trans е_trans р_trans т_t


Before:		iPad
TrueAfter	а_trans й_trans п_trans а_trans д_trans
PredAfter	д_trans э_trans т_trans е_trans р_trans т_trans е_trans р_trans 

In [47]:
torch.save(transformer_model.state_dict(), BACKUP_PREFFIX + 'transformer.mdl')

## Часть 3. [2 балла]  Дополнительные признаки
Предложите и покажите, как можно было бы повысить качество нейросетевых моделей. Примерные варианты:
1. ансамблирование нейронных сетей
2. добавление морфологоческих признаков 
3. использование эмбеддингов слов 


* Можно подавать на вход часть речи слова
* Обрабатывать не только символьно, но и предложения целиком. Есть частые ошибки при склонении
* Предобработать текст. Например, сделать все буквы маленькими(lowercase)
* При ансамблировании сетей можно при одинаковых длинах усреднить вероятности классов. При разных выбрать что-то одно

## Часть 4. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Проведите анализ ошибок: когда модель ошибается? Можно ли скзаать, почему модель ошибается? Сравните результаты всех разработанных моделей. Что помогло вам в выполнении работы, чего не хватало?

Мы обучили 5 моделей, которые все в принципе показвают неплохой результат.
* Первая модель была обычная seq2seq Encoder-Decoder, все последующие уже использовали attention. Можно заметить, что даже самый простой attention слой -- DotProductAttention, повышает результаты. Мы использовали в сумме 4 разных подхода.
* Последней моделью стал Transformer -- более современный подход для работы с последовательностями

Хочется добавить, что из-за что, библиотека allennlp очень высокоуровнева, мы не можем как-то менять внутреннюю архтектуру модели. Нам кажется, что добавив отдельный выход, который предсказывает класс слова, можно добиться очень высоких результатов. Например, если мы определили, что слово алглийское, то можно исправить некоторые опечатки сети в преффиксах, а если мы натнулись на PLAIN и результат сети отличается в 1-2х символах, то скорее всего наша сеть "опечаталась" и нужно просто скопировать вход. К тому же сеть лучше научится различать классы из-за дополнительного лосса

Модель ошибается в склонениях, потому что не видит контекста слова. Без него даже человек не смотжет правильно нормализовать текст. Иногда без контекста непонятно, что перед нами за объект.

В классе PLAIN ошибок не так много, поэтому мы и выводим только те, что при нормализации меняются

Лучший результат показала модель seq2seq с мультипликативным вниманием